## Install Dependencies

In [2]:
# !pip install PyPDF2
# !pip install gensim
# !pip install nltk
# !pip install WordCloud
# !pip install pdfminer3
# !pip install mlflow
# !pip3 install spacy
# !pip3 install pyLDAvis==2.1.2

In [3]:
# !pip freeze

---
## Import Libraries

In [39]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

# pd.options.display.max_columns = 50
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 50)

plt.rcParams.update({'figure.figsize':(15,6), 'figure.dpi':60})
plt.style.use('fivethirtyeight')

/Users/xinminaw/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [40]:
# import sys  
# import os
# sys.path.append('src') 
# from edge import *

In [41]:
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this

# PDF text extraction
from pdfminer3.layout import LAParams, LTTextBox
from pdfminer3.pdfpage import PDFPage
from pdfminer3.pdfinterp import PDFResourceManager
from pdfminer3.pdfinterp import PDFPageInterpreter
from pdfminer3.converter import PDFPageAggregator
from pdfminer3.converter import TextConverter

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

# Others
import requests
import io
import string
import re
from pprint import pprint
from tqdm.notebook import tqdm
import io
import json


### Loading Language Model

In [42]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

import spacy
#spacy.cli.download("en_core_web_sm")
nlp = spacy.load("en_core_web_sm", disable=['ner'])

[nltk_data] Downloading package punkt to /Users/xinminaw/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/xinminaw/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Preparing Stop Words

In [43]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

from sklearn.feature_extraction import text
stop_words = text.ENGLISH_STOP_WORDS.union(stop_words)

## Report Details

Enter required information about the report.  Report must be in PDF format and downloadable from a URL.

1. Company Name
2. Company Ticker Symbol
3. Year of the Report
4. URL of the Report

In [6]:
DATA_FOLDER = "../data/"
report_url_excel = pd.ExcelFile(DATA_FOLDER + "ESG Data Collection.xlsx")
asianbanks_df = report_url_excel.parse('Asian Banks (Group 18)')
assetmanagers_df = report_url_excel.parse('Asset managers (Group 7)')
insurance_df = report_url_excel.parse('Insurance (Group 20)')
pensionfunds_df = report_url_excel.parse('Pension funds (Group 8)')
assetmanagers_df.head()

/Users/xinminaw/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/Users/xinminaw/opt/anaconda3/lib/python3.8/site-packages/xlrd/xlsx.py:266: DeprecationWarning: This method will be removed in future versions.  Use 'tree.iter()' or 'list(tree.iter())' instead.
  for elem in self.tree.iter() if Element_has_iter else self.tree.getiterator():
/Users/xinminaw/opt/anaconda3/lib/python3.8/site-packages/xlrd/xlsx.py:266: DeprecationWarning: This method will be removed in future versions.  Use 'tree.iter()' or 'list(tree.iter())' instead.
  for elem in self.tree.iter() if Element_has_iter else self.tree.getiterator():


,Company,Market,Link,Year,Remarks,Unnamed: 5
0,BlackRock,Regional,https://www.blackrock.com/corporate/literature...,2020,NaN,NaN
1,NaN,Regional,https://www.blackrock.com/corporate/literature...,2020,NaN,NaN
2,NaN,Regional,https://www.blackrock.com/corporate/literature...,2019,NaN,NaN
3,NaN,Regional,https://www.blackrock.com/corporate/literature...,2018,NaN,NaN
4,NaN,Regional,https://www.blackrock.com/corporate/literature...,2018-2019,NaN,NaN


In [7]:
asianbanks_df.shape[0] + assetmanagers_df.shape[0] + insurance_df.shape[0] + pensionfunds_df.shape[0]
76/400

/Users/xinminaw/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0.19

In [8]:
assetmanagers_df.Company = assetmanagers_df.Company.fillna(method='ffill')
pensionfunds_df.Fund = pensionfunds_df.Fund.fillna(method='ffill')
asianbanks_df.Company = asianbanks_df.Company.fillna(method='ffill')
insurance_df.Company = insurance_df.Company.fillna(method='ffill')
assetmanagers_df.head()

/Users/xinminaw/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Company,Market,Link,Year,Remarks,Unnamed: 5
0,BlackRock,Regional,https://www.blackrock.com/corporate/literature...,2020,NaN,NaN
1,BlackRock,Regional,https://www.blackrock.com/corporate/literature...,2020,NaN,NaN
2,BlackRock,Regional,https://www.blackrock.com/corporate/literature...,2019,NaN,NaN
3,BlackRock,Regional,https://www.blackrock.com/corporate/literature...,2018,NaN,NaN
4,BlackRock,Regional,https://www.blackrock.com/corporate/literature...,2018-2019,NaN,NaN


## Text Extraction
Extract information from the PDF report.  

In [44]:
def extract_pdf(file, verbose=False):
    """
    Process raw PDF text to structured and processed PDF text to be worked on in Python.

    Parameters
    ----------
    file : textfile
        Textfile that contains raw PDF text.

    Return
    ------
    text : str
        processed PDF text if no error is throw

    """  
    
    if verbose:
        print('Processing {}'.format(file))

    try:
        resource_manager = PDFResourceManager()
        fake_file_handle = io.StringIO()
        codec = 'utf-8'
        laparams = LAParams()

        converter = TextConverter(resource_manager, fake_file_handle, codec=codec, laparams=laparams)
        page_interpreter = PDFPageInterpreter(resource_manager, converter)
        
        password = ""
        maxpages = 0
        caching = True
        pagenos = set()

        content = []

        for page in PDFPage.get_pages(file,
                                      pagenos, 
                                      maxpages=maxpages,
                                      password=password,
                                      caching=True,
                                      check_extractable=False):

            page_interpreter.process_page(page)

            content.append(fake_file_handle.getvalue())

            fake_file_handle.truncate(0)
            fake_file_handle.seek(0)        

        text = '##PAGE_BREAK##'.join(content)

        # close open handles
        converter.close()
        fake_file_handle.close()
        
        return text

    except Exception as e:
        print(e)

        # close open handles
        converter.close()
        fake_file_handle.close()

        return ""

In [29]:
def extract_content(url):
    """
    Downloads PDF text content from a given URL and parse PDF to obtain processed text.

    Parameters
    ----------
    url : str
        String that contains url to desired PDF

    Return
    ------
    text : str
        processed PDF text if no error is throw

    """   
    headers={"User-Agent":"Mozilla/5.0"}

    try:
        # retrieve PDF binary stream
        r = requests.get(url, allow_redirects=True, headers=headers)
        
        # access pdf content
        text = extract_pdf(io.BytesIO(r.content))

        # return concatenated content
        return text

    except:
        return ""

/Users/xinminaw/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Data Collection
- must be online PDF, cannot have links that require PDF downloads

In [ ]:
# pension funds
for index,row in assetmanagers_df.iterrows():
    report_company = row["Company"]
    report_year = str(row["Year"])
    report_url = row["Link"]
    report_content = extract_content(report_url)
    report = {'company':report_company, 'year':report_year,'url':report_url, 'content':report_content}
    with open(DATA_FOLDER + "sustainability_reports/pension_funds/" + report_company+report_year+'.json', "w") as outfile:  
        json.dump(report, outfile)
        
# asset managers
for index,row in assetmanagers_df.iterrows():
    report_company = row["Company"]
    report_year = str(row["Year"])
    report_url = row["Link"]
    report_content = extract_content(report_url)
    report = {'company':report_company, 'year':report_year,'url':report_url, 'content':report_content}
    with open(DATA_FOLDER + "sustainability_reports/asset_managers/" + report_company+report_year+'.json', "w") as outfile:  
        json.dump(report, outfile)

# insurance       
for index,row in insurance_df.loc[76:,].iterrows():
    report_company = row["Company"]
    report_year = "" if  str(row["Year"]) == np.nan else str(row["Year"])
    report_url = row["PDF Link"]
    report_content = extract_content(report_url)
    report = {'company':report_company, 'year':report_year,'url':report_url, 'content':report_content}
    with open(DATA_FOLDER + "sustainability_reports/insurance/" + report_company+report_year+'.json', "w") as outfile:  
        json.dump(report, outfile)

# asian banks
for index,row in asianbanks_df.iterrows():
    report_company = row["Company"]
    report_year = "" if  str(row["Year"]) == np.nan else str(row["Year"])
    report_url = row["Link"]
    report_content = extract_content(report_url)
    report = {'company':report_company, 'year':report_year,'url':report_url, 'content':report_content}
    with open(DATA_FOLDER + "sustainability_reports/asian_banks/" + report_company+report_year+'.json', "w") as outfile:  
        json.dump(report, outfile)

/Users/xinminaw/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


---

## Natural Language Processing

### Extracting content by pages and sentences

In [11]:
def preprocess_lines(line_input):
    """
    Helper Function to preprocess and clean sentences from raw PDF text 

    Parameters
    ----------
    line_input : str
        String that contains a sentence to be cleaned

    Return
    ------
    line : str
        Cleaned sentence

    """      
    # removing header number
    line = re.sub(r'^\s?\d+(.*)$', r'\1', line_input)
    # removing trailing spaces
    line = line.strip()
    # words may be split between lines, ensure we link them back together
    line = re.sub(r'\s?-\s?', '-', line)
    # remove space prior to punctuation
    line = re.sub(r'\s?([,:;\.])', r'\1', line)
    # ESG contains a lot of figures that are not relevant to grammatical structure
    line = re.sub(r'\d{5,}', r' ', line)
    # remove emails
    line = re.sub(r'\S*@\S*\s?', '', line)
    # remove mentions of URLs
    line = re.sub(r'((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*', r' ', line)
    # remove multiple spaces
    line = re.sub(r'\s+', ' ', line)
    # join next line with space
    line = re.sub(r' \n', ' ', line)
    line = re.sub(r'.\n', '. ', line)
    line = re.sub(r'\x0c', ' ', line)
    
    return line

/Users/xinminaw/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
def remove_non_ascii(text):
    """
    Helper Function to remove non ascii characters from text
    """
    printable = set(string.printable)
    return ''.join(filter(lambda x: x in printable, text))

def not_header(line):
    """
    Helper Function to remove headers
    """
    return not line.isupper()

def extract_pages_sentences(nlp, text):    
    """
    Extracting text from raw PDF text and store them by pages and senteces. Raw text is also cleand by removing junk, URLs, etc.
    Consecutive lines are also grouped into paragraphs and spacy is used to parse sentences.
    Parameters
    ----------
    nlp: spacy nlp model
        NLP model to parse sentences
    text : str
        Raw PDF text

    Return
    ------
    pages_content : list of str
        A list containing text from each page of the PDF report. Page number is the index of list + 1
    
    pages_sentences : list of list
        A list containing lists. Page number is the index of outer list + 1. Inner list contains sentences from each page
 
    """  
    MIN_WORDS_PER_PAGE = 500
    
    pages = text.split('##PAGE_BREAK##')
    #print('Number of Pages: {}'.format(len(pages)))

    lines = []
    for i in range(len(pages)):
        page_number = i + 1
        page = pages[i]
        
        # remove non ASCII characters
        text = remove_non_ascii(page)
        
        # if len(text.split(' ')) < MIN_WORDS_PER_PAGE:
        #     print(f'Skipped Page: {page_number}')
        #     continue
        
        prev = ""
        for line in text.split('\n\n'):
            # aggregate consecutive lines where text may be broken down
            # only if next line starts with a space or previous does not end with dot.
            if(line.startswith(' ') or not prev.endswith('.')):
                prev = prev + ' ' + line
            else:
                # new paragraph
                lines.append(prev)
                prev = line

        # don't forget left-over paragraph
        lines.append(prev)
        lines.append('##SAME_PAGE##')
        
    lines = '  '.join(lines).split('##SAME_PAGE##')
    
    # clean paragraphs from extra space, unwanted characters, urls, etc.
    # best effort clean up, consider a more versatile cleaner
    
    pages_content = []
    pages_sentences = []

    for line in lines[:-1]: # looping through each page
        
        line = preprocess_lines(line)       
        pages_content.append(str(line).strip())

        sentences = []
        # split paragraphs into well defined sentences using spacy
        for part in list(nlp(line).sents):
            sentences.append(str(part).strip())

        #sentences += nltk.sent_tokenize(line)
            
        # Only interested in full sentences and sentences with 10 to 100 words. --> filter out first page/content page
        sentences = [s for s in sentences if re.match('^[A-Z][^?!.]*[?.!]$', s) is not None]
        sentences = [s.replace('\n', ' ') for s in sentences]
        
        pages_sentences.append(sentences)
        
    return pages_content, pages_sentences #list, list of list where page is index of outer list


/Users/xinminaw/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Data preprocessing 
1. Lowercase
2. Lemmatization

In [13]:
def preprocessing(report):
    """
    Lemmatize,lowercase and remove stopwords for pages of a report
    
    Parameters
    ----------
    report: list of str
        A list containing text from each page of the PDF report. Page number is the index of list + 1

    Return
    ------
    report_pages : list of str
        A list containing processed text from each page of the PDF report. Page number is the index of list + 1
    
    """  
    
    report_pages = []

    def para_to_sent(para):
        """
        Helper function to split paragraphs into well defined sentences using spacy
        """
        sentences = []
        for part in list(nlp(para).sents):
            sentences.append(str(part).strip())
        return sentences

    def remove_stopwords(texts):
        """
        Helper function to remove stopwords from sentence
        """
        return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]


    def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
        """https://spacy.io/api/annotation"""
        """
        Helper function to lemmatize text in sentence
        """
        texts_out = []
        doc = nlp(texts) 
        texts_out.append(" ".join([token.lemma_ for token in doc]))
        return texts_out
    
    for page in report:

        sentences = para_to_sent(page.lower())

        # Do lemmatization keeping only noun, adj, vb, adv
        page_data = []
        for sentence in sentences : 
            data_lemmatized = lemmatization(sentence, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
            page_data.extend(data_lemmatized)
        page_para_lemma = "".join(page_data)
        
        report_pages.append(page_para_lemma)
    
    return report_pages



/Users/xinminaw/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
# all_reports_pages_sentences_preprocessed = [] # list of dictionaries, each dictionary is 1 report
# for report in all_reports_pages_sentences:
#     report_pages_preprocessed = preprocessing(report["report_pages"])
#     report_sentences_preprocessed = [preprocessing(page) for page in report["report_sentences"]]
#     report["report_pages_preprocessed"] = report_pages_preprocessed 
#     report["report_sentences_preprocessed"] = report_sentences_preprocessed 
#     all_reports_pages_sentences_preprocessed.append(report)

# # pipeline
# #report_preprocessed = preprocessing(report_pages) 

/Users/xinminaw/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Filtering Report

1. If the page contains at least 3 words from relevant_terms_directFilter_lem, keep the page, else filter it out.
2. If the page contains any word from (relevant_terms_combinationC_lem AND relevant_terms_combinationA_lem) OR (relevant_terms_combinationC_lem AND  relevant_terms_combinationB_lem), keep the page, else filter it out.




In [15]:
def lemmatization(text_list, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    # lemmatize text in sentence
    """https://spacy.io/api/annotation"""
    texts_out = []
    for texts in text_list:
        texts = texts.lower()
        texts_out.append(" ".join([token.lemma_ for token in nlp(texts)]))
    return texts_out

/Users/xinminaw/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [36]:
relevant_terms_directFilter = set(["carbon","co2","environment","GHG emissions","Greenhouse Gas","carbon footprint","carbon emissions","Scope 1","Scope 2",
                               "Scope 3", "WACI","Carbon Intensity","carbon pricing","net-zero","metrics and targets","TCFD",
                                "sustainability goals","decarbonisation","climate",'energy', 'emission', 'emissions', 'renewable', 'carbon', 'fuel', 'power', 
                               'green', 'gas', 'green energy', 'sustainable', 'climate', 'sustainability', 'environmental', 'environment', 'GHG', 
                               'decarbon', 'energy consumption', 'paper consumption','water consumption', 'carbon intensity', 'waste management', 'electricity consumption', 
                                'cdp', 'global warming', 'business travel','climate solutions', 'decarbonization', 'cvar', 'climate value-at-risk','waste output'])
relevant_terms_combinationA = ["emissions","exposure","carbon related","esg","sustainable","green","climate sensitive","impact investing", "investment framework", 'msci', 'ftse', 'responsible investing', 'responsible investment','transition']
relevant_terms_combinationB = ["portfolio","assets","AUM","investment","financing","ratings","revenue","bond","goal","insurance", "equity", "swap", "option", "portfolio holdings", "risk management",'financial products']
relevant_terms_combinationC = ["net zero","carbon footprint","CO2","carbon","oil","coal", "gas", "fossil fuel","green"] # compare [CB or CA]  compare with ABC
relevant_terms_combination_directFilter_lem = lemmatization(relevant_terms_directFilter)
relevant_terms_combinationA_lem = lemmatization(relevant_terms_combinationA)
relevant_terms_combinationB_lem = lemmatization(relevant_terms_combinationB)
relevant_terms_combinationC_lem = lemmatization(relevant_terms_combinationC)
#relevant_terms_wordEmbeddings = [""] 

/Users/xinminaw/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [17]:
def filter_report_highLevel(report):
    """
    Page filter to filter report for only relevant pages with decarbonisation related words.
    Two types of word filters: direct and indirect. Direct contains words that are directly related to decarbonisation while indirect contains other relevant decarbonisation information.
    
    Parameters
    ----------
    report: list of str
        A list containing text from each page of the PDF report. Page number is the index of list + 1

    Return
    ------
    filtered_report_direct : dict of {int : str}
        A dictionary that contains relevant pages obtained using direct filter. The key is the page number and value is the text on the page. 
    
    filtered_report_indirect : dict of {int : str}
        A dictionary that contains relevant pages obtained using indirect filter. The key is the page number and value is the text on the page.     
    """  
    filtered_report_direct = {}
    filtered_report_indirect = {}
    for i in range(len(report)):
        page = report[i]
        page_number = i + 1
        no_words = len(page.split(" "))
        if sum(map(page.__contains__, relevant_terms_combination_directFilter_lem)) > 2:
            filtered_report_direct[page_number] = page
        elif (any(map(page.__contains__, relevant_terms_combinationA_lem)) and any(map(page.__contains__, relevant_terms_combinationC_lem))) or (any(map(page.__contains__, relevant_terms_combinationB_lem)) and any(map(page.__contains__, relevant_terms_combinationC_lem))):
            filtered_report_indirect[page_number] = page
    return filtered_report_direct,filtered_report_indirect


/Users/xinminaw/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
# filter digits
def is_number(string): 
    """
    Helper function that checks if a string contains numbers
    """
    test_str = string
    # next() checking for each element, reaches end, if no element found as digit
    res = True if next((chr for chr in test_str if chr.isdigit()), None) else False
    return res

def is_date(string):
    """
    Helper function that checks if a string contains dates such as "september"/"Monday at 12:01am"/1999 etc
    """
    if re.match('.*([1-2][0-9]{3})', string) != None:
        return True
    return False

def filter_report_numbers(filtered_report): 
    """
    Page filter to filter the filtered report for pages with numbers that are not dates as to track decarbonisation progress, we need numbers.
    
    Parameters
    ----------
    filtered_report: dict of {int : str}
        A dictionary that contains relevant pages obtained using page filters. The key is the page number and value is the text on the page.
        
    Return
    ------
    filtered_report_numbers : dict of {int : str}
        A dictionary that contains relevant pages with numbers. The key is the page number and value is the text on the page. 
    
    """ 
        
    filtered_report_numbers = {}
    for page_number,page in filtered_report.items():
        # remove all dates from page first
        page_no_date = " ".join([word for word in page.split(" ") if is_date(word) == False])
        # retain pages with numbers
        if is_number(page_no_date):
            filtered_report_numbers[page_number] = page           
    return filtered_report_numbers

/Users/xinminaw/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [19]:
def filter_tables(filtered_report):    
    """
    Page filter to filter the filtered report for pages with tables to aid table detection in downstream tasks. This is done by checking if page contains at least 10 numbers + 1 units
    
    Parameters
    ----------
    filtered_report: dict of {int : str}
        A dictionary that contains relevant pages with numbers. The key is the page number and value is the text on the page.
        
    Return
    ------
    filtered_report_numbers : dict of {int : str}
        A dictionary that contains relevant pages with tables. The key is the page number and value is the text on the page. 
    
    """ 
    
    
    units = ['tonnes', 'tons', 'kwh', 'kg', 'kilogram', 'kilowatt hour', 'gigajoules', 'gj', 'litre', 'liter', 'co2e', 'tco2e', 'tco2', 'mwh', 'megawatt hour', 'gwh', 'gigawatt hour', '%', 'cubic metres', 'cm3', 'm3', 'per employee','co2']
    filtered_report_numbers = {}
    for page_number,page in filtered_report.items():
        no_numbers = 0
        units_flag = 0
        for word in page.split(" "):
            try: 
                float(word)
                if is_date(word) == False:
                    no_numbers += 1
            except: 
                if any(char.isdigit() for char in word):
                    no_numbers += 1
        for unit in units:
            if unit in page:
                units_flag += 1     
        if (no_numbers >= 10) & (units_flag >= 1):
            filtered_report_numbers[page_number] = page
    return filtered_report_numbers

/Users/xinminaw/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Preprocessing all FIs and saving output files

In [20]:
folder_names = ["asset_managers","pension_funds","insurance","asian_banks"]
empty_json = 0
for fi in folder_names: 
    # reload in data to preprocess
    all_reports = [file for file in os.listdir(DATA_FOLDER + f"sustainability_reports/{fi}/") if file[-4:] == "json"]
    all_reports_list = []
    for file in all_reports:
         with open(DATA_FOLDER + f"sustainability_reports/{fi}/" + file,) as inputfile:
                report = json.load(inputfile)
                # filter out empty pages
                if report["content"] == "":
                    empty_json += 1
                else : 
                    all_reports_list.append(report)
    
    # preprocessing
    all_reports_pages_sentences = [] # list of dictionaries, each dictionary is 1 report
    for report in all_reports_list:
        report_pages, report_sentences = extract_pages_sentences(nlp, report['content'])
        report["report_pages"] = report_pages
        report["report_sentences"] = report_sentences
        all_reports_pages_sentences.append(report)
    
        
    all_reports_pages_sentences_preprocessed = [] # list of dictionaries, each dictionary is 1 report
    for report in all_reports_pages_sentences:
        report_pages_preprocessed = preprocessing(report["report_pages"])
        report_sentences_preprocessed = [preprocessing(page) for page in report["report_sentences"]]
        report["report_pages_preprocessed"] = report_pages_preprocessed 
        report["report_sentences_preprocessed"] = report_sentences_preprocessed 
        all_reports_pages_sentences_preprocessed.append(report)
     
    all_reports_pages_sentences_preprocessed_filtered = []
    for report in all_reports_pages_sentences_preprocessed:
        # filter by both words and numbers
        # pages
        filtered_report_direct_highLevel,filtered_report_indirect_highLevel = filter_report_highLevel(report["report_pages_preprocessed"])
        filtered_report_pages_direct_numbers = filter_report_numbers(filtered_report_direct_highLevel)
        filtered_report_pages_indirect_numbers = filter_report_numbers(filtered_report_indirect_highLevel)
        report["filtered_report_pages_direct"] = filtered_report_pages_direct_numbers
        report["filtered_report_pages_indirect"] = filtered_report_pages_indirect_numbers
        index_direct = [page_no-1 for page_no in filtered_report_pages_direct_numbers.keys()] 
        index_indirect = [page_no-1 for page_no in filtered_report_pages_indirect_numbers.keys()] 

        # sentences
        filtered_report_sentences_direct_numbers = {}
        for page in filtered_report_pages_direct_numbers.keys():
            filtered_report_sentences_direct_numbers[page] = report["report_sentences_preprocessed"][page-1]
        filtered_report_sentences_indirect_numbers = {}
        for page in filtered_report_pages_indirect_numbers.keys():
            filtered_report_sentences_indirect_numbers[page] = report["report_sentences_preprocessed"][page-1]
        report["filtered_report_sentences_direct"] = filtered_report_sentences_direct_numbers
        report["filtered_report_sentences_indirect"] = filtered_report_sentences_indirect_numbers

        # tables
        report["filtered_report_tables_direct"] = filter_tables(filtered_report_pages_direct_numbers)
        report["filtered_report_tables_indirect"] = filter_tables(filtered_report_pages_indirect_numbers)
        all_reports_pages_sentences_preprocessed_filtered.append(report)
        
    with open(DATA_FOLDER + "preprocessed_sustainability_reports/" + f'all_{fi}_preprocessed_vFINAL.json', "w") as outfile:  
        json.dump(all_reports_pages_sentences_preprocessed_filtered, outfile)

print(empty_json)
    

/Users/xinminaw/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


77


### Reading In Pre-processed Data
-- Skip all previous sections

In [31]:
with open(DATA_FOLDER + "preprocessed_sustainability_reports/" + 'all_pensionfunds_preprocessed.json',) as inputfile:  
    all_pensionfunds_preprocessed = json.load(inputfile)

/Users/xinminaw/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
